# Ordering


## Warning: This notebook will place live orders

Use a paper trading account (during market hours).


In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=13)
# util.logToConsole()

<IB connected to 127.0.0.1:7497 clientId=13>

Create a contract and a market order:

In [3]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)
ib.reqMktData(contract, '', False, False)
diff = 2e-2
TPBuy = 1.0885
SLBuy = 1.085
lotsize = 1000
print("Buy Signal Found...")
order = MarketOrder('BUY', lotsize)
stopLoss = StopOrder('SELL',lotsize, SLBuy)
takeProfit = LimitOrder('SELL', lotsize, TPBuy)
print(ib.placeOrder(contract, order))
ib.placeOrder(contract, stopLoss)
ib.placeOrder(contract, takeProfit)
print("Buy Order Placed.")

Buy Signal Found...
Trade(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=MarketOrder(orderId=256, clientId=13, action='BUY', totalQuantity=1000), orderStatus=OrderStatus(orderId=256, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 6, 5, 11, 8, 17, 41570, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')
Buy Order Placed.


In [4]:
TPSell = 1.085
SLSell  = 1.0891

order = MarketOrder('SELL', lotsize)
stopLoss = StopOrder('BUY',lotsize, SLSell)
takeProfit = LimitOrder('BUY', lotsize, TPSell)
ib.placeOrder(contract, order)
ib.placeOrder(contract, stopLoss)
ib.placeOrder(contract, takeProfit)

Trade(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=LimitOrder(orderId=261, clientId=13, action='BUY', totalQuantity=1000, lmtPrice=1.085), orderStatus=OrderStatus(orderId=261, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 6, 5, 11, 8, 43, 369426, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [21]:
trade = ib.placeOrder(contract, order)

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [10]:
ib.sleep(1)
trade.log

[TradeLogEntry(time=datetime.datetime(2024, 6, 4, 13, 15, 39, 664878, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 4, 13, 15, 39, 798448, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 4, 13, 15, 39, 828478, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='Fill 100.0@1.08635', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 4, 13, 15, 39, 828478, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)]

``trade`` will also available from ``ib.trades()``:

In [15]:
assert trade in ib.trades()


[]


Likewise for ``order``:

In [12]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [16]:
limitOrder = LimitOrder('BUY', 30, 1.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

Trade(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=LimitOrder(orderId=35, clientId=13, action='BUY', totalQuantity=30, lmtPrice=1.05), orderStatus=OrderStatus(orderId=35, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 6, 4, 13, 18, 0, 647234, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

``status`` will change from "PendingSubmit" to "Submitted":

In [20]:
ib.sleep(1)
#assert limitTrade.orderStatus.status == 'Submitted'

print(ib.openTrades())
def count_opened_trades():
    account_summary = ib.accountSummary()
    eur_balance = None
    for item in account_summary:
        if item.tag == 'CashBalance' and item.currency == 'EUR':
            eur_balance = float(item.value)
    if eur_balance > 5000 or eur_balance < 4000:
        return 1
    else:
        return 0

count_opened_trades()



[]


0

In [43]:
import pandas_ta as ta
import pandas as pd

def ema_signal(df, current_candle, backcandles):
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df.iloc[start:end]
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0
    
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0

def get_bars(durationStr,barSizeSetting):
    bars = ib.reqHistoricalData(contract, endDateTime='', durationStr=durationStr,barSizeSetting=barSizeSetting, whatToShow='MIDPOINT', useRTH=True, formatDate=1,keepUpToDate=True)
    return bars

def get_candles_frame(durationStr,barSizeSetting):
    bars = get_bars(durationStr,barSizeSetting)
    dfstream = pd.DataFrame(columns=['date','open','high','low','close'])
    for i in range(len(bars)):
        dfstream.loc[i,'date'] = bars[i].date
        dfstream.loc[i,'open'] = bars[i].open
        dfstream.loc[i,'high'] = bars[i].high
        dfstream.loc[i,'low'] = bars[i].low
        dfstream.loc[i,'close'] = bars[i].close
    dfstream["ATR"] = ta.atr(dfstream.high, dfstream.low, dfstream.close, length=7)
    dfstream["EMA_fast"]=ta.ema(dfstream.close, length=30)
    dfstream["EMA_slow"]=ta.ema(dfstream.close, length=50)
    dfstream['RSI']=ta.rsi(dfstream.close, length=10)
    my_bbands = ta.bbands(dfstream.close, length=15, std=1.5)
    dfstream=dfstream.join(my_bbands) 
    dfstream['TotalSignal'] = dfstream.apply(lambda row: total_signal(dfstream, row.name, 7), axis=1)
    dfstream.set_index('date', inplace=True)
    dfstream.index = dfstream.index.tz_convert('Europe/Paris')
    return dfstream

pd.set_option('display.max_rows', None)
df = get_candles_frame('1 D','5 mins')
df.tail(10)



BarData(date=datetime.datetime(2024, 6, 4, 10, 35, tzinfo=backports.zoneinfo.ZoneInfo(key='US/Eastern')), open=1.08703, high=1.087065, low=1.08677, close=1.086985, volume=-1.0, average=-1.0, barCount=-1)

In [31]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [32]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

Trade(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=LimitOrder(orderId=18, clientId=13, permId=1579293976, action='BUY', totalQuantity=30.0, lmtPrice=0.1, auxPrice=0.0), orderStatus=OrderStatus(orderId=18, status='Submitted', filled=0.0, remaining=30.0, avgFillPrice=0.0, permId=1579293976, parentId=0, lastFillPrice=0.0, clientId=13, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 503252, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 617408, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 641088, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 33, 353698, tzinfo=datetime.timezone.utc), statu

And now cancel it:

In [33]:
ib.cancelOrder(limitOrder)

Trade(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=LimitOrder(orderId=18, clientId=13, permId=1579293976, action='BUY', totalQuantity=30.0, lmtPrice=0.1, auxPrice=0.0), orderStatus=OrderStatus(orderId=18, status='PendingCancel', filled=0.0, remaining=30.0, avgFillPrice=0.0, permId=1579293976, parentId=0, lastFillPrice=0.0, clientId=13, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 503252, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 617408, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 641088, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 33, 353698, tzinfo=datetime.timezone.utc), s

In [34]:
limitTrade.log

[TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 503252, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 617408, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 22, 641088, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 33, 353698, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 33, 463276, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 33, 483651, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0),
 TradeLogEntry(time=datetime.datetime(2024, 6, 3, 15, 56, 45, 234321, tzinfo=datetime.timezone.utc), s

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [35]:
%%time
order = MarketOrder('BUY', 30)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

Wall time: 139 ms


What are our positions?

In [36]:
ib.positions()

[Position(account='DU9038516', contract=Forex('EURUSD', conId=12087792, localSymbol='EUR.USD', tradingClass='EUR.USD'), position=30.0, avgCost=1.1725835944143037)]

What's the total of commissions paid today?

In [44]:
sum(fill.commissionReport.commission for fill in ib.fills())

9.2176

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [43]:
order = MarketOrder('SELL', 30)
trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()
ib.whatIfOrder(contract, order)

OrderState(status='PreSubmitted', initMarginBefore='1.7976931348623157E308', maintMarginBefore='1.7976931348623157E308', equityWithLoanBefore='1.7976931348623157E308', initMarginChange='1.7976931348623157E308', maintMarginChange='1.7976931348623157E308', equityWithLoanChange='1.7976931348623157E308', initMarginAfter='1.7976931348623157E308', maintMarginAfter='1.7976931348623157E308', equityWithLoanAfter='1.7976931348623157E308', commission=1.7976931348623157e+308, minCommission=1.7976931348623157e+308, maxCommission=1.7976931348623157e+308, commissionCurrency='', warningText='Important Note: Your order size is below the EUR 20000 IdealPro minimum.\nYou may cancel and increase the order size if you wish to route to IdealPro\nor select the Transmit button to route as an odd lot order.\nNote that odd lot orders are not guaranteed executions at the IdealPro displayed quotes.', completedTime='', completedStatus='')

In [118]:
ib.disconnect()